In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from random import randint

In [2]:
POPULATION_SIZE = None
INDIVIDUAL_LENGTH = None
CROSSOVER_RATE = None
MUTATION_RATE = None

generation_count = None
population = None
fitness_values = None
probs = None
best_value = None
best_individual = None
cities = None
cost_matrix = None

In [3]:
def init():
    global POPULATION_SIZE, INDIVIDUAL_LENGTH, CROSSOVER_RATE, MUTATION_RATE, generation_count, population, fitness_values, best_value, best_individual, cities, cost_matrix
    POPULATION_SIZE = 30
    INDIVIDUAL_LENGTH = 734
    CROSSOVER_RATE = 0.9
    MUTATION_RATE = 0.01
    
    generation_count = 200
    population = np.zeros((POPULATION_SIZE, INDIVIDUAL_LENGTH)).astype(int)
    file = open("data/uy734.tsp", "r")
#     cities = np.zeros((INDIVIDUAL_LENGTH,2))
    cities = []
    lines_skip = 7
    best_value = 1e9
    for i in file.read().split('\n')[lines_skip:lines_skip + INDIVIDUAL_LENGTH]:
        cities.append([float(i.split()[1]), float(i.split()[2])])
    cost_matrix = np.zeros((INDIVIDUAL_LENGTH, INDIVIDUAL_LENGTH))
    for i in range(INDIVIDUAL_LENGTH):
        for j in range(INDIVIDUAL_LENGTH):
            cost_matrix[i,j] = np.linalg.norm(np.array(cities[i]) - np.array(cities[j]))

In [4]:
def init_population():
    global POPULATION_SIZE, INDIVIDUAL_LENGTH, population
    individual = np.arange(INDIVIDUAL_LENGTH)
    np.random.shuffle(individual)

    for i in range(POPULATION_SIZE):
        np.random.shuffle(individual)
        idx = np.argmin(individual)
        population[i] = individual

In [5]:
def evaluate():
    global population, INDIVIDUAL_LENGTH, POPULATION_SIZE, fitness_values, probs, best_value, best_individual
    fitness_values = []
    probs = []
    for i in range(population.shape[0]):
        fitness = 0
        for j in range(population.shape[1]):
            if(j == 0):
                continue
            fitness += cost_matrix[int(population[i,j]), int(population[i,j-1])]
        fitness += cost_matrix[int(population[i,0]), int(population[i,INDIVIDUAL_LENGTH-1])]
        fitness_values.append(fitness)
    
    if(best_value > np.min(fitness_values)):
        best_value = np.min(fitness_values)
        best_individual = population[np.argmin(fitness_values)]
    
    probs = fitness_values / np.sum(fitness_values)
    probs = [[value,key] for (value,key) in sorted([(value,key) for (key,value) in enumerate(probs)])]
    for i in range(population.shape[0]):
        if(i==0):
            continue
        probs[i][0] += probs[i-1][0]
    return None

In [6]:
#sequential constructive crossover
def SCX(P1,P2,val):
    global INDIVIDUAL_LENGTH
    offspring = []
    offspring.append(0)
    
    mask = np.ones(INDIVIDUAL_LENGTH).astype(int)
    mask[0] = 0
    
    next_dp1 = np.zeros(INDIVIDUAL_LENGTH).astype(int)
    next_dp2 = np.zeros(INDIVIDUAL_LENGTH).astype(int)
    
    curr = (np.where(P1 == 0)[0][0] + val)%INDIVIDUAL_LENGTH
    next_dp1[0] = P1[curr]
    
    while(P1[curr] != 0):
        idx = np.where(P1 == P1[curr])[0][0]
        idx = (idx+val)%INDIVIDUAL_LENGTH
        next_dp1[P1[curr]] = P1[idx]
        curr = idx
        
    curr = (np.where(P2 == 0)[0][0] + val)%INDIVIDUAL_LENGTH
    next_dp2[0] = P2[curr]
    while(P2[curr] != 0):
        idx = np.where(P2 == P2[curr])[0][0]
        idx = (idx+val)%INDIVIDUAL_LENGTH
        next_dp2[P2[curr]] = P2[idx]
        curr = idx

    while(len(offspring) < INDIVIDUAL_LENGTH):
        last_node = offspring[-1]
        dp1 = last_node
        dp2 = last_node 
        while(mask[int(dp1)] == 0):
            dp1 = next_dp1[int(dp1)]
        while(mask[int(dp2)] == 0):
            dp2 = next_dp2[int(dp2)]
        if(cost_matrix[int(last_node), int(dp1)] <= cost_matrix[int(last_node), int(dp2)]):
            offspring.append(int(dp1))
            mask[int(dp1)] = 0
        else:
            offspring.append(dp2)
            mask[int(dp2)] = 0
    return offspring

In [7]:
#partially mapped crossover
def PMX(P1,P2):
    global INDIVIDUAL_LENGTH
    offspring1 = P1.copy()
    offspring2 = P2.copy()
    
    next_dp1 = np.zeros(INDIVIDUAL_LENGTH).astype(int)
    next_dp2 = np.zeros(INDIVIDUAL_LENGTH).astype(int)
    
    for idx, val in enumerate(P1):
        next_dp1[int(val)] = idx
    for idx, val in enumerate(P2):
        next_dp2[int(val)] = idx
    
    m = randint(0, INDIVIDUAL_LENGTH)
    
    for index in range(m, int(m +INDIVIDUAL_LENGTH/2)):
        i = index%INDIVIDUAL_LENGTH
        idx = next_dp1[P2[i]]
        offspring1[i], offspring1[idx] = offspring1[idx], offspring1[i]
    
    for index in range(m, int(m +INDIVIDUAL_LENGTH/2)):
        i = index%INDIVIDUAL_LENGTH
        idx = next_dp2[P1[i]]
        offspring2[i], offspring2[idx] = offspring2[idx], offspring2[i]
    
    return offspring1, offspring2

In [8]:
def mutate(individual, prob):
    global INDIVIDUAL_LENGTH, population
    if(prob >= 0.5):
        m = INDIVIDUAL_LENGTH
        n = 0
        while(n < m or m ==0):
            m = randint(1, INDIVIDUAL_LENGTH)
            n = randint(1, INDIVIDUAL_LENGTH)
        individual[m:n] = individual[m:n][::-1]
    else:
        m = INDIVIDUAL_LENGTH
        n = 0
        while(n <= m or m ==0):
            m = randint(1, INDIVIDUAL_LENGTH)
            n = randint(1, INDIVIDUAL_LENGTH)
        clone = individual.copy()
        individual[1:(n-m+1)], individual[(n-m+1):n] = clone[m:n], clone[1:m]
    return individual

In [9]:
def mutation():
    global population, MUTATION_RATE, POPULATION_SIZE
    for i in range(POPULATION_SIZE):
        if(i==0):
            continue
        if(np.random.sample() < MUTATION_RATE):
            population[i] = mutate(population[i].copy(),np.random.sample())


In [10]:
def crossover():
    global population, POPULATION_SIZE, CROSSOVER_RATE
    crossover_list = []
    for i in range(POPULATION_SIZE):
        if(np.random.sample() < CROSSOVER_RATE):
            crossover_list.append(i)
    np.random.shuffle(crossover_list)
    for i in range(0, len(crossover_list)-1, 2):
        idx1 = crossover_list[i]
        idx2 = crossover_list[i+1]
        population[idx1], population[idx2] = SCX(population[idx1].copy(), population[idx2].copy(), 1), SCX(population[idx1].copy(), population[idx2].copy(), -1)
#         population[idx1], population[idx2] = PMX(population[idx1].copy(), population[idx2].copy())
    

In [11]:
def select(prob):
    for i in range(POPULATION_SIZE):
        if(probs[i][0] >= prob):
            return population[probs[i][1]]
    return population[probs[POPULATION_SIZE-1][1]]

In [12]:
def selection():
    global population, probs, POPULATION_SIZE
    parents = np.zeros((POPULATION_SIZE, INDIVIDUAL_LENGTH)).astype(int)
    init_count = int(POPULATION_SIZE/2)
    parents[0:init_count] = population[[key for [value,key] in probs[POPULATION_SIZE-init_count:POPULATION_SIZE]]]
    for i in range(init_count, POPULATION_SIZE):
        parents[i] = select(np.random.sample())
    population = parents

In [13]:
init()
init_population()

In [ ]:
generation_count = 500
for i in range(generation_count):
    print("Generation", i+1)
    evaluate()
    print("Best value generation", np.min(fitness_values))
    print("Best value", best_value)
    selection()
    crossover()
    mutation()


Generation 1
Best value generation 1591074.9589762639
Best value 1591074.9589762639
Generation 2
Best value generation 1220191.1912127335
Best value 1220191.1912127335
Generation 3
Best value generation 976802.2262507004
Best value 976802.2262507004
Generation 4
Best value generation 844606.540959509
Best value 844606.540959509
Generation 5
Best value generation 756163.8631853666
Best value 756163.8631853666
Generation 6
Best value generation 692287.0694815841
Best value 692287.0694815841
Generation 7
Best value generation 641761.080973813
Best value 641761.080973813
Generation 8
Best value generation 618655.6843777806
Best value 618655.6843777806
Generation 9
Best value generation 567300.9015150486
Best value 567300.9015150486
Generation 10
Best value generation 512930.1426885871
Best value 512930.1426885871
Generation 11
Best value generation 473738.25270169077
Best value 473738.25270169077
Generation 12
Best value generation 431964.61553254514
Best value 431964.61553254514
Generatio

Generation 99
Best value generation 105370.65552374396
Best value 105370.65552374396
Generation 100
Best value generation 109946.26908834523
Best value 105370.65552374396
Generation 101
Best value generation 109318.14298321435
Best value 105370.65552374396
Generation 102
Best value generation 109380.66107577531
Best value 105370.65552374396
Generation 103
Best value generation 100643.10016289631
Best value 100643.10016289631
Generation 104
Best value generation 105231.57422717547
Best value 100643.10016289631
Generation 105
Best value generation 107604.63620211146
Best value 100643.10016289631
Generation 106
Best value generation 108259.57132284292
Best value 100643.10016289631
Generation 107
Best value generation 103528.60946526354
Best value 100643.10016289631
Generation 108
Best value generation 105734.78536281484
Best value 100643.10016289631
Generation 109
Best value generation 106448.51364633364
Best value 100643.10016289631
Generation 110
Best value generation 107775.65002565159

Generation 196
Best value generation 100647.55223533353
Best value 95686.4412718369
Generation 197
Best value generation 96858.29244593708
Best value 95686.4412718369
Generation 198
Best value generation 101144.41396981078
Best value 95686.4412718369
Generation 199
Best value generation 99874.72875037877
Best value 95686.4412718369
Generation 200
Best value generation 99288.03619490814
Best value 95686.4412718369
Generation 201
Best value generation 102467.08949880993
Best value 95686.4412718369
Generation 202
Best value generation 100142.5015800651
Best value 95686.4412718369
Generation 203
Best value generation 97596.62426821611
Best value 95686.4412718369
Generation 204
Best value generation 96068.93423839474
Best value 95686.4412718369
Generation 205
Best value generation 99730.69170679327
Best value 95686.4412718369
Generation 206
Best value generation 98489.5474687535
Best value 95686.4412718369
Generation 207
Best value generation 99821.10297761997
Best value 95686.4412718369
Ge

Generation 295
Best value generation 98591.00686515123
Best value 94728.55161230736
Generation 296
Best value generation 96262.33367946684
Best value 94728.55161230736
Generation 297
Best value generation 92072.5741093225
Best value 92072.5741093225
Generation 298
Best value generation 95744.3743272442
Best value 92072.5741093225
Generation 299
Best value generation 98341.59885960654
Best value 92072.5741093225
Generation 300
Best value generation 99040.29186321585
Best value 92072.5741093225
Generation 301
Best value generation 98387.37558785631
Best value 92072.5741093225
Generation 302
Best value generation 94994.21807687361
Best value 92072.5741093225
Generation 303
Best value generation 98250.79132659707
Best value 92072.5741093225
Generation 304
Best value generation 97741.21947371983
Best value 92072.5741093225
Generation 305
Best value generation 93190.51402806301
Best value 92072.5741093225
Generation 306
Best value generation 99965.34529657691
Best value 92072.5741093225
Gene

In [ ]:
print(best_value)
plot_cities = np.array(cities)
plt.plot(plot_cities[:,0], plot_cities[:,1], 'bo')
for i in range(INDIVIDUAL_LENGTH):
    if(i == 0):
        city_idx_prev = int(best_individual[0])
        city_idx = int(best_individual[INDIVIDUAL_LENGTH-1])
        plt.plot(plot_cities[(city_idx_prev,city_idx),0], plot_cities[(city_idx_prev,city_idx),1], 'r-')
        continue
    city_idx_prev = int(best_individual[i-1])
    city_idx = int(best_individual[i])
    plt.plot(plot_cities[(city_idx_prev,city_idx),0], plot_cities[(city_idx_prev,city_idx),1], 'r-')
    
plt.show()